In [1]:
# importing libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

# Machine learning packages
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

## OLD CODE

In [ ]:
# reading data from csv and creating a dataframe
df = pd.read_csv('Google-Playstore.csv')

# dataframe dimensions
print(f"This dataframe has {df.shape[0]} rows and {df.shape[1]} columns.")

In [ ]:
df.info()

In [ ]:
#Drop unnecessary variables: Deveopler Website, Developer Id, Developer Email, Privacy Policy
df = df.drop(['App Name','App Id','Minimum Installs','Maximum Installs','Minimum Android','Last Updated','Developer Id', 'Developer Website', 'Developer Email', 'Privacy Policy','Content Rating','Privacy Policy','Ad Supported','In App Purchases','Editors Choice'], axis = 1)

In [ ]:
df.info()

In [ ]:
df['Installs'] = df['Installs'].str.replace('+','')
df['Installs'] = df['Installs'].str.replace(',','')
df.Installs = pd.to_numeric(df.Installs,errors='coerce')
df.Installs=df.Installs.apply(func=np.log1p)

In [ ]:
# remove apps other than US
df = df[df['Currency'] == 'USD']
df = df.drop(['Currency'],axis=1)

In [ ]:
#Remove Varies with Device from Size
df = df[df['Size'] != 'Varies with device']
kb_index = df.Size[df.Size.str.contains('k')].index
mb_index = df.Size[(df.Size.str.contains('M'))].index
#Remove M,k,, from the size variable
#df['Size'] = df['Size'].str.replace('.','')
df['Size'] = df['Size'].str.replace(',','')
df.Size = df.Size.str.replace('M','')
df.Size = df.Size.str.replace('k','')
df.Size = pd.to_numeric(df.Size,errors='coerce')
df.Size.loc[mb_index] = df.Size.loc[mb_index] * 1000

In [ ]:
df.Released=pd.to_datetime(df.Released)
df['ReleaseMonth']= df['Released'].dt.month
df = df.dropna(subset=['ReleaseMonth'])

In [ ]:
df = df[(df.Rating>=1) & (df.Rating<=5)]

In [ ]:
df.info()

In [ ]:
df.corr(method='pearson')

In [ ]:
df=pd.get_dummies(df,columns=['Category','Free'],dtype=np.int64)


In [ ]:
print(df.hist(['Rating','Rating Count','Size','Installs','Price'],figsize=(12,8),xlabelsize=12,ylabelsize=12))

In [ ]:
df.Price=df.Price.apply(func=np.log1p)
df.Rating=df.Rating.apply(func=np.log1p)
df['Rating Count']=df['Rating Count'].apply(func=np.log1p)
df.Size=df.Size.apply(func=np.log1p)

In [ ]:
df.info()

In [ ]:
print(df.columns)

In [ ]:
X = df.drop(['Installs','Released'],axis=1)

In [ ]:
y = df['Installs']

In [ ]:
model = sm.OLS(y.astype(float), X.astype(float)).fit()
predictions = model.predict(X) 

print_model = model.summary()
print(print_model)

In [ ]:
#After first iteration
tobedropped = ['Installs','Released','Category_Sports','Category_Social','Category_Shopping','Category_Productivity','Category_Lifestyle','Category_Communication']

In [ ]:
X = df.drop(tobedropped,axis=1)
y= y = df['Installs']
model = sm.OLS(y.astype(float), X.astype(float)).fit()
predictions = model.predict(X) 

print_model = model.summary()
print(print_model)

In [ ]:
## Importing from EDA

In [ ]:
df = pd.read_csv('data.csv')

In [ ]:
df.info()

## NEW CODE

In [2]:
# reading data from csv and creating a dataframe
df = pd.read_csv('data.csv')

# dataframe dimensions
print(f"This dataframe has {df.shape[0]} rows and {df.shape[1]} columns.")

This dataframe has 399284 rows and 31 columns.


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 399284 entries, 0 to 399283
Data columns (total 31 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   Unnamed: 0        399284 non-null  int64  
 1   App Name          399283 non-null  object 
 2   App Id            399284 non-null  object 
 3   Category          399284 non-null  object 
 4   Rating            399284 non-null  float64
 5   Rating Count      399284 non-null  float64
 6   Installs          399284 non-null  float64
 7   Minimum Installs  399284 non-null  float64
 8   Maximum Installs  399284 non-null  int64  
 9   Free              399284 non-null  bool   
 10  Price             399284 non-null  float64
 11  Currency          399284 non-null  object 
 12  Size              399284 non-null  float64
 13  Minimum Android   398998 non-null  object 
 14  Released          399284 non-null  object 
 15  Last Updated      399284 non-null  object 
 16  Content Rating    39

In [4]:
df = df.drop(['Unnamed: 0','Currency','Released','App Name','App Id','Category','Minimum Installs','Maximum Installs','Minimum Android','Last Updated','Content Rating','Ad Supported','In App Purchases','Editors Choice'], axis = 1)

In [5]:
X = df.drop(['Installs'],axis=1)
y = df['Installs']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size = 0.25,random_state = 345)


In [6]:
model = sm.OLS(y.astype(float), X.astype(float)).fit()
predictions = model.predict(X) 

print_model = model.summary()
print(print_model)

                                 OLS Regression Results                                
Dep. Variable:               Installs   R-squared (uncentered):                   0.986
Model:                            OLS   Adj. R-squared (uncentered):              0.986
Method:                 Least Squares   F-statistic:                          2.053e+06
Date:                Sun, 29 Nov 2020   Prob (F-statistic):                        0.00
Time:                        23:49:56   Log-Likelihood:                     -6.0820e+05
No. Observations:              399284   AIC:                                  1.216e+06
Df Residuals:                  399270   BIC:                                  1.217e+06
Df Model:                          14                                                  
Covariance Type:            nonrobust                                                  
                      coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------